# Image Super Resolution #1
<hr style='height:5px'>
kwanki.ahn<br>
2019/06/08(Sun) skeleton<br>
refer : https://github.com/titu1994/Image-Super-Resolution 
<hr style='height:5px'>



In [1]:
from __future__ import print_function, division

from keras.models import Model
from keras.layers import Concatenate, Add, Average, Input, Dense, Flatten, BatchNormalization, Activation, LeakyReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, Convolution2DTranspose
from keras import backend as K
from keras.utils.np_utils import to_categorical
import keras.callbacks as callbacks
import keras.optimizers as optimizers

Using TensorFlow backend.


In [3]:
import img_utils
import numpy as np
import os
import time
import warnings

image dataset source : https://data.vision.ee.ethz.ch/cvl/DIV2K/<br>

Let's start with<br>
(NTIRE 2017) Low Res Images:<br>
- Train Data Track 2 unknown downgrading operators x2 (LR images)<br>
- Validation Data Track 2 unknown downgrading operators x2 (LR images)<br>

High Resolution Images:<br>
- Train Data (HR images)<br>
- Validation Data (HR images)<br>

In [4]:
# !wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_unknown_X2.zip
# !wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_unknown_X2.zip
# !wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
# !wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip

In [5]:
train_path = img_utils.output_path
validation_path = img_utils.validation_output_path
path_X = img_utils.output_path + "X/"
path_Y = img_utils.output_path + "y/"

In [8]:
train_path, validation_path, path_X, path_Y

('/home/jupyter/image_sr_dataset/train_images/train/',
 '/home/jupyter/image_sr_dataset/train_images/validation/',
 '/home/jupyter/image_sr_dataset/train_images/train/X/',
 '/home/jupyter/image_sr_dataset/train_images/train/y/')

In [10]:
def PSNRLoss(y_true, y_pred):
    """
    PSNR is Peek Signal to Noise Ratio, which is similar to mean squared error.
    It can be calculated as
    PSNR = 20 * log10(MAXp) - 10 * log10(MSE)
    When providing an unscaled input, MAXp = 255. Therefore 20 * log10(255)== 48.1308036087.
    However, since we are scaling our input, MAXp = 1. Therefore 20 * log10(1) = 0.
    Thus we remove that component completely and only compute the remaining MSE component.
    """
    return -10. * K.log(K.mean(K.square(y_pred - y_true))) / K.log(10.)

def psnr(y_true, y_pred):
    assert y_true.shape == y_pred.shape, "Cannot calculate PSNR. Input shapes not same." \
                                         " y_true shape = %s, y_pred shape = %s" % (str(y_true.shape),
                                                                                   str(y_pred.shape))

    return -10. * np.log10(np.mean(np.square(y_pred - y_true)))

In [ ]:
init = Input(shape=shape)
